In [238]:
from __future__ import annotations
import pandas as pd
import csv

Data before cleanup:

In [239]:
pd.read_csv('./data/DataSample.csv')

,_ID,TimeSt,Country,Province,City,Latitude,Longitude
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010
3,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730
4,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.37160,-80.97730
...,...,...,...,...,...,...,...
22020,5614801,2017-06-21 12:23:07.880,CA,ON,Saint Catharines,43.16440,-79.24560
22021,5614888,2017-06-21 08:23:01.793,CA,AB,Calgary,51.02093,-114.10621
22022,5614909,2017-06-21 00:23:07.903,CA,ON,Whitby,43.88730,-78.94220
22023,5614912,2017-06-21 11:23:07.953,CA,ON,Oakville,43.49340,-79.71260


Place the data into a dictionary to filter it for duplicates

In [240]:
def filter_data(file: str) -> dict[str, list]:
    """Returns a dictionary of the data with the suspicious entries filtered out"""
    raw_data = pd.read_csv(file)
    seen = {}
    
    for index, row in raw_data.iterrows():
        formatted = f'{row[" TimeSt"]}, {row["Latitude"]} {row["Longitude"]}'
        
        if formatted in seen:
            seen[formatted] = []
        else:
            seen.setdefault(formatted, []).append(row)
    return seen

After filtering the data, write it to a new file

In [241]:
def cleanup(file: str) -> None:
    """Filter the data from <file> and write it to a new file"""
    filtered_data = filter_data(file)
    
    with open('./data/FilteredDataSample.csv', 'w') as f:
        csv_writer = csv.writer(f)
        
        for key in filtered_data:
            if filtered_data[key]:
                for row in filtered_data[key]:
                    csv_writer.writerow(row)

In [242]:
cleanup('./data/DataSample.csv')

Data after cleanup:

In [243]:
pd.read_csv('./data/FilteredDataSample.csv')

,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123
0,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.9399,-81.2709
1,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.5776,-80.2201
2,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.3716,-80.9773
3,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.3716,-80.9773
4,4516693,2017-06-21 14:00:00.597,CA,ON,Kitchener,43.4381,-80.5099
...,...,...,...,...,...,...,...
17967,5614689,2017-06-21 13:23:01.370,CA,ON,London,42.9611,-81.2431
17968,5614801,2017-06-21 12:23:07.880,CA,ON,Saint Catharines,43.1644,-79.2456
17969,5614909,2017-06-21 00:23:07.903,CA,ON,Whitby,43.8873,-78.9422
17970,5614912,2017-06-21 11:23:07.953,CA,ON,Oakville,43.4934,-79.7126
